# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [ ]:
!pip3 install wget

In [1]:
# Importing relevant packages
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium Mouseposition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df = pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [4]:
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant columns: 'Launch Site', 'Lat(Latitude)', 'Long(Longitude)', 'class'
launch_site_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_site_df = df.groupby(['Launch Site'], as_index=False).first()
launch_site_df = launch_site_df[['Launch Site', 'Lat', 'Long']]
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with
# a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius = 1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(nasa_coordinate,
                          # Create an icon as a text label
                          icon = DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0),
                                        html='<div style="font-size: 12; color: #d35400;"><b>%s</b></div>'%'NASA JSC',
                                        )
                          )

site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

An example of folium.Circle:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [8]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate 
# (Lat, Long) values. In addition, add Launch site name as a popup label

# Initialize folium feature Group to contain the markers and circles
incidents = folium.map.FeatureGroup()

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup lab
for lat, long, ls in zip(launch_site_df.Lat, launch_site_df.Long, launch_site_df['Launch Site']):
    circle = folium.Circle((lat, long), raius=500, color='red',
                  fill=True).add_child(folium.Popup(ls))
    marker = folium.map.Marker((lat, long), icon=DivIcon(icon_size=(20,20),
                                                              icon_anchor=(0,0),
                                                              html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>'% ls,))
    incidents.add_child(circle)
    incidents.add_child(marker)

site_map.add_child(incidents)


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast? - Yes

Also please try to explain your findings.

# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [11]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

C:\Users\ASENPR~1\AppData\Local\Temp/ipykernel_14076/1374732283.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`

In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker((record['Lat'], record['Long']), icon=folium.Icon(color='white',icon_color=record['marker_color']))
                                                              #html='<div style="font-size:12; color:#d35400;"><b>%s</b></div>'% ls,))
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'
mouse_position = MousePosition(
        position = 'topright',
        separator=' Long: ',
        empty_string='NaN',
        lng_first=False,
        num_digits=20,
        prefix=' Lat: ',
        lat_formatter=formatter,
        lng_formatter=formatter,
    )

site_map.add_child(mouse_position)
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lat2, lon1, lon2):
    # approximate radius of earth in km
    R = 6373.0
    
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [17]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coastal_lat = 28.56368
coastal_long = -80.56738

launch_site_lat = 28.56344
launch_site_long = -80.5768

distance_coastline2 = calculate_distance(launch_site_lat, coastal_lat, launch_site_long, coastal_long)
distance_coastline2

0.9206433366515538

In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
        (coastal_lat, coastal_long),
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline2),
        )
    )

site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = ((launch_site_lat, launch_site_long), (coastal_lat, coastal_long))
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)
site_map

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

A railway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>

A highway map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>

A city map symbol may look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>

In [19]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
coastal_coordinates = [(28.56346, -80.56788), (28.56346, -80.56788), (28.5804, -80.6419), (34.63315, -120.62658)]
railway_coordinates = [(34.63363, -120.62478), (28.5731, -80.65397), (28.57225, -80.58525), (28.57225, -80.58525)]
city_coordinates = [(34.63865, -120.45839), (28.60743, -80.81131), (28.40129, -80.60411), (28.40129, -80.60411)]
highway_coordinates = [(34.63801, -120.4565), (28.5734, -80.65546), (28.56317, -80.57075), (28.56317, -80.57075)]


# reversing the coordinates for railway, highway and city to match the order in the dataframe
start = len(coastal_coordinates) - 1
railway_coordinates = [railway_coordinates[i] for i in range(start, -1, -1)]
city_coordinates = [city_coordinates[i] for i in range(start, -1, -1)]
highway_coordinates = [highway_coordinates[i] for i in range(start, -1, -1)]


In [21]:
calculate_distance(launch_site_lat, coastal_coordinates[0][0], launch_site_long, coastal_coordinates[0][1])

0.8714141045965847

In [22]:
def distance_marker_func(coordinates, distance):
    distance_marker = folium.Marker((coordinates),
                                    icon=DivIcon(icon_size = (20,20),
                                                icon_anchor=(0,0),
                                                html = "<div style = 'font-size:12; color:#0000FF;'><b>%s</b></div>" % "{:10.2f} KM".format(distance),
                                                )
                                   )
    return distance_marker


In [23]:
def lines_marker(coords, w=1):
    lines = folium.PolyLine(locations=coords, weight=w)
    return lines

In [24]:
new_df = launch_site_df

In [25]:
new_df['Coastal'] = coastal_coordinates
new_df['Railway'] = railway_coordinates
new_df['Highway'] = highway_coordinates
new_df['City'] = city_coordinates
new_df

,Launch Site,Lat,Long,Coastal,Railway,Highway,City
0,CCAFS LC-40,28.562302,-80.577356,"(28.56346, -80.56788)","(28.57225, -80.58525)","(28.56317, -80.57075)","(28.40129, -80.60411)"
1,CCAFS SLC-40,28.563197,-80.576820,"(28.56346, -80.56788)","(28.57225, -80.58525)","(28.56317, -80.57075)","(28.40129, -80.60411)"
2,KSC LC-39A,28.573255,-80.646895,"(28.5804, -80.6419)","(28.5731, -80.65397)","(28.5734, -80.65546)","(28.60743, -80.81131)"
3,VAFB SLC-4E,34.632834,-120.610745,"(34.63315, -120.62658)","(34.63363, -120.62478)","(34.63801, -120.4565)","(34.63865, -120.45839)"


In [26]:
lines_ls = []
distance_marks = []
distance_values = []
for ls, lat, long, coastal, railway, highway, city in zip(new_df['Launch Site'], new_df.Lat, new_df.Long, new_df.Coastal, new_df.Railway, new_df.Highway, new_df.City):
    
    colors = ['#0000FF', '#DEB887', '#20B2AA', '#C71585'] # initialize colors
    
    # calculate the various distances using the 'calculate_distance func'
    coastal_distance = calculate_distance(lat, coastal[0], long, coastal[1])
    railway_distance = calculate_distance(lat, railway[0], long, railway[1])
    highway_distance = calculate_distance(lat, highway[0], long, highway[1])
    city_distance = calculate_distance(lat, city[0], long, city[1])
    
    distance_values.append((ls,[coastal_distance, railway_distance,  highway_distance, city_distance]))
    
    # draw the distance markers
    coastal_distance_marker = distance_marker_func(coastal, coastal_distance)
    railway_distance_marker = distance_marker_func(railway, railway_distance)
    highway_distance_marker = distance_marker_func(highway, highway_distance)
    city_distance_marker = distance_marker_func(city, city_distance)
    
    # draw line markers
    coastal_line_marker = lines_marker(((lat,long),(coastal[0], coastal[1])))
    railway_line_marker = lines_marker(((lat, long), (railway)))
    highway_line_marker = lines_marker(((lat, long), (highway[0], highway[1])))
    city_line_marker = lines_marker(((lat, long), (city[0], city[1])))
    
    # populate distance_marks list
    distance_marks.append(coastal_distance_marker)
    distance_marks.append(railway_distance_marker)
    distance_marks.append(highway_distance_marker)
    distance_marks.append(city_distance_marker)
    
    
    lines_ls.append(coastal_line_marker)
    lines_ls.append(railway_line_marker)
    lines_ls.append(highway_line_marker)
    lines_ls.append(city_line_marker)

#site_map.add_child(distance_marks[0])
#site_map.add_child(line_markers[0])
#site_map
    

In [28]:
# draw distance markers and distance lines on site map
for distance_mark, line in zip(distance_marks, lines_ls):
    site_map.add_child(distance_mark)
    site_map.add_child(line)

In [29]:
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.
